<img src="https://terrabrasilis.dpi.inpe.br/queimadas/portal/design-system/files/LogoINPEQmdMed.png" align="right" width="150"/>

# <left><span style="color:#cf4c39">BIG Love Data Day - Script 01
# <left><span style="color:#cf4c39">Uso de imagens de satélite para análise espaço-temporal das mudanças na cobertura da paisagem devido a ciclos agrícolas e ocorrência de queimadas</span></left>
<hr style="border:2px solid #e69627;">
<br/>
<div style="text-align: center;font-size: 90%;">
    <b style="font-size: 120%;">Paulo Cunha e Fabiano Morelli</b>
    <br/><br/>
    Programa Queimadas, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contatos: <a href="mailto:paulo.cunha@inpe.br">paulo.cunha@inpe.br </a> e <a href="mailto:fabiano.morelli@inpe.br">fabiano.morelli@inpe.br</a> 
    <br/><br/>
    Ultíma Atualização: 03 de Junho de 2025
</div>

 <br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;">   
<b>Resumo</b> Este Jupyter Notebook é parte do material do Love Data Day organizado pela Base de Informações Georeferenciadas (BIG) do Instituto Nacional de Pesquisas Espaciais (INPE). Nele é demonstrado como utilizar o serviço SpatioTemporal Asset Catalog (STAC) para consultar dados disponíveis no catálogo do instituto, bem como a análise espaço-temporal de mudanças na cobertura da paisagem devido  a ocorrência de queimadas utilizando imagens do satélite Sentinel-2.

# <left><span style="color:#cf4c39"> 🛰️ Introdução
<hr style="border:1px solid #e69627;">

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">   Os incêndios florestais têm se tornado cada vez mais frequentes e severos — não apenas em regiões tradicionalmente propensas ao fogo, mas também em áreas inesperadas, como diversos biomas brasileiros, incluindo a Amazônia, Pantanal e o Cerrado. Compreender o comportamento e os impactos desses incêndios é essencial para o desenvolvimento de estratégias eficazes.</div>
<br>
<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">
Nesta primeira parte, serão utilizados dados de sensoriamento remoto obtidos pelo satélite Sentinel-2, processados a partir do Brazil Data Cube (BIG) e acessados por meio do serviço STAC (SpatioTemporal Asset Catalog), para estimar a extensão da área afetada por um incêndio em áreas de agrícolas utilizadas para lavouras e pastagens no Oeste do Estado de São Paulo no período de 19 a 24 de agosto de 2024. Este exemplo de análise é fundamental para identificar padrões espaciais e temporais de queima, contribuindo para uma compreensão mais abrangente dos impactos.</div>

# <left><span style="color:#cf4c39"> Escolha da Área para Anáslise -  "Dia do Fogo 🔥 - SP"
<hr style="border:1px solid #e69627;">

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;"> O <b>"Dia do Fogo em São Paulo"</b>, foi um evento ocorrido em <b>23 de agosto de 2024</b>, muito noticiado em repercussão nacinal pelos principais meios de comunicação e informação pública. Foi marcado por um aumento significativo na quanidade de focos de queimadas e incêndios, especialmente na porção Oeste do Estado. Neste caso de estudo foi analiada uma área próxima da cidade de Votuporanga.</br>
<img src='assets/mapa_votuporanga.png' width="1000">
</div><br>

**Notícias na Imprensa** 📰 [Dia do Fogo SP](https://www.google.com/search?sca_esv=8f5d8c7e0081b1d2&rlz=1C1FCXM_pt-PTBR994BR994&q=dia+do+fogo+sp&tbm=nws&source=lnms&fbs=AEQNm0AuaLfhdrtx2b9ODfK0pnmi046uB92frSWoVskpBryHTpm4Flwlr5cHTE9P1oWvlAZ_BMzBfkHx_sz6KVHYmpoBgg_r2U_G8VRtklQCu_dWWjcOucNh5nM4o1m1i2GqXHmOPZWc3KnoK2vnkM1e72_pWpwdw-vJnJvGiT1hVDcJHgJrQqNDw9wy12nTLeMWHJwEx8CL5L3av-B6eG9gxDWVdQuA-g&sa=X&sqi=2&ved=2ahUKEwjPub702PqJAxVVp5UCHS9_BbYQ0pQJegQIERAB&biw=1280&bih=551&dpr=1.5')

<img src='assets/noticias.png' width="1000" >

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">
Nesta seção, vamos explorar a coleta e o processamento dos dados de imagens da coleção do Sentinel-2, antes e depois do incêndio, utilizando os dados disponibilizados pelo Brazil Data Cube (BIG).</div>
<br>
<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">
Os dados do Sentinel-2 são adequados para esse tipo de análise devido sua alta frequência de revisita e por fornecem 12 bandas espectrais com resolução de 10 a 60 metros, permitindo o mapeamento de cicatrizes de queimadas.</div>

# <left><span style="color:#cf4c39">Importando as bibliotecas </span></left>
<hr style="border:1px solid #e69627;">

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Image
from skimage.morphology import (
    binary_closing,
    binary_erosion,
    binary_dilation,
    remove_small_objects,
    remove_small_holes,
    disk
)
from shapely.geometry import box
import folium
import warnings
warnings.filterwarnings("ignore")

import rasterio
from rasterio import Affine
from rasterio.crs import CRS
#import rasterio.transform
from rasterio.windows import from_bounds
from rasterio.warp import Resampling, reproject, transform


In [ ]:
# Verifica se a pasta de saída dos arquivos existe
folder_name = './output'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)  # Cria a pasta


# <left><span style="color:#cf4c39">Abrindo um shapefile com a área de interesse (AOI)</span></left>
<hr style="border:1px solid #e69627;">

In [ ]:
# Leitura do arquivo shapefile e conversão para o formato Geopandas
shapefile = "./data/queimadas/area_interesse.shp"
aoi = gpd.read_file(shapefile)

# Apresenta o conteúdo do Geopandas
aoi

#### Apresenta a AOI em um mapa dinâmico com uma imagem de satélite no fundo

In [ ]:
f = folium.Figure(width=1000, height=400)

mapa = folium.Map(location=[aoi.CNTY, aoi.CNTX],
               zoom_start=10,
               tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
               attr="Esri World Imagery"
        ).add_to(f)

folium.GeoJson(aoi).add_to(mapa)

mapa

### Extrair informações a partir do objeto geográfico do shapefile 
A partir do shapefile, vamos criar o bounding box
```python
xmin, ymin, xmax, ymax = aoi.total_bounds
bbox = (xmin, ymin, xmax, ymax)
print(bbox)
```
Caso sua AOI não possua um atributo mostrando as coordenadas do centro, é possível calcular um objeto geográfico que representa o centro do poligono

```python
centroide = aoi.geometry.union_all().centroid
```

Com este objeto temos um ponto e podemos utilizar as propriedades X e Y para retornar o valor das coordenadas de Longitude e latitude respectivamente

# <left><span style="color:#cf4c39">👩🏽‍💻 STAC Client API
<hr style="border:1px solid #e69627;">

Para acessar as funcionalidades, importa-se o pacote `pystac_client`:

In [ ]:
import pystac_client
pystac_client.__version__

Para isso utilizaremos o serviço Spatio Temporal Asset Catalog (STAC) por meio de um client na linguagem de programação Python.

O endereço do serviço STAC do BDC é https://data.inpe.br/bdc/stac/v1/. 

In [ ]:
servico  = "https://data.inpe.br/bdc/stac/v1/"
catalogo  = pystac_client.Client.open(servico)

# <left><span style="color:#cf4c39">🔍 Buscando as Imagens Pré-Fogo (19/08/2024) e Pós-Fogo (24/08/2024) utilizando o serviço STAC
<hr style="border:1px solid #e69627;">

Vamos escolher a coleção do Sentinel-2,o range de datas e a porcentagem da cobertura de nuvens para análise, partir do bounding box ou pelo tile_id

### Busca pelo Bounding Box

Será utilizado o Bounding Box extraído diretamente da AOI. No entanto é possível passar uma tupla conforme o exemplo abaixo
```python
bbox= (-49.87930174,-20.37185787,-49.53239264,-20.03585787)
```

### Parametros de entrada

In [ ]:
collection = ["S2_L2A-1"]
date_range = "2024-08-18/2024-08-25"
filter = { "eo:cloud_cover": {"lt": 20} } # Filtro para máxima cobertura de nuvens

In [ ]:
# Realizando a busca com os filtros
item_search = catalogo.search(bbox=aoi.total_bounds,
    collections=[collection],
    query=filter, 
    datetime=date_range,  # Intervalo de datas
)

In [ ]:
item_search.item_collection()

In [ ]:
items = list(item_search.item_collection())


# Ordenar os itens por data (crescente: menor para maior)
items = sorted(items, key=lambda x: x.datetime)

### É possível utilizar as propriedades de cada item para exibir ou solecionar o(s) objeto(s) de uma coleção selecionada

In [ ]:
# Itera sobre os itens e exibe detalhes

for i, item in enumerate(items):
    item_id = item.id  # ID do item para identificação

    date = item.properties.get('datetime', 'N/A')  # Data da aquisição
    
    cloud_cover = item.properties.get('eo:cloud_cover', 'N/A')  # Obtém a cobertura de nuvens
    
    tile_id = items[0].properties.get('tileId')
        
    print(f"Imagem [{i}]: {item_id}, Data: {date}, Cobertura de nuvens: {cloud_cover:.1f}% , Tile_ID: {tile_id}")

--------

### Busca pelo Tile Id do Sentinel-2

```python
tile_s2_bdc = '22KFC'
collection = ["S2_L2A-1"]
date_range = "2024-08-18/2024-08-25"
filters = {
    "bdc:tiles": {"eq": tile_s2_bdc},  # Filtro pelo tile específico
    "eo:cloud_cover": {"lt": 100}  # Filtro para cobertura
}
```

In [ ]:
imagem_pre = items[0].assets
imagem_pos = items[2].assets

In [ ]:
imagem_pre

In [ ]:
imagem_pos

In [ ]:
from IPython.display import display, HTML

# Dados dos thumbnails
thumbnail_pre = imagem_pre['PVI']
thumbnail_pos = imagem_pos['PVI']

# Datas e títulos
data_pre = items[0].properties.get('datetime', 'N/A')[:10]
data_pos = items[2].properties.get('datetime', 'N/A')[:10]

titulo_pre = f"Imagem pré-fogo - data: {data_pre}"
titulo_pos = f"Imagem pós-fogo - data: {data_pos}"

# HTML com títulos e imagens lado a lado
html_code = f"""
<div style="display: flex; align-items: flex-start;">
    <div style="text-align: center; margin-right: 20px;">
        <div style="font-weight: bold; margin-bottom: 5px;">{titulo_pre}</div>
        <img src="{thumbnail_pre.href}">
    </div>
    <div style="text-align: center;">
        <div style="font-weight: bold; margin-bottom: 5px;">{titulo_pos}</div>
        <img src="{thumbnail_pos.href}">
    </div>
</div>
"""

display(HTML(html_code))


## Função para a redução da resolução espacial de 10m para 20m

In [ ]:
def transforme_20m (asset, transforme, crs):
    '''Reamostragem de dados de 20 para 10 mts'''

    tranform_20 = Affine(20.0, 0.0, transforme.c,
                         transforme.d,-20.0,transforme.f)

    # Cria uma matriz numpy (grade regular) para o resultado reamostrado, com as dimensões (shape) e o tipo corretos
    array_20 = np.zeros((int(asset.shape[0]/2), int(asset.shape[1]/2)), dtype=asset.dtype)

    reproject(
        source=asset.data, # Passa apenas os valores e não o masked array
        destination=array_20,
        src_transform=transforme,
        src_crs=crs,
        dst_transform=tranform_20,
        dst_crs=crs,
        src_nodata=0, # Assume 0 como nodata para os dados de entrada
        dst_nodata=0, # Assume 0 como nodata para os dados de saída
        resampling=Resampling.nearest
    )
    return array_20

In [ ]:
ds = rasterio.open(imagem_pre['B04'].href)
ds.crs

## Função pra leitura das bandas nas imagens pré o pós

In [ ]:
def read(uri: str, bbox: list, masked: bool = True, crs: str = None):
    """Read raster window as numpy.ma.masked_array."""
    source_crs = CRS.from_string('EPSG:4326')
    
    if crs:
        source_crs = CRS.from_string(crs)

    # Expects the bounding box has 4 values
    w, s, e, n = bbox
        
    with rasterio.open(uri) as dataset:
        transformer = transform(source_crs, dataset.crs, [w, e], [s, n])
        window = from_bounds(transformer[0][0], transformer[1][0], 
                             transformer[0][1], transformer[1][1], dataset.transform)
        
        box_trasform = rasterio.transform.from_bounds(transformer[0][0], transformer[1][0],
                                                      transformer[0][1], transformer[1][1], window.width, window.height)
        
        return dataset.read(1, window=window, masked=masked)*0.0001, box_trasform

## Leitura das bandas na imagem pré

In [ ]:
# ler imagens pré fogo
b04_pre,b04_pre_transf = read(imagem_pre['B04'].href, bbox=aoi.total_bounds)
b08_pre,b08_pre_transf = read(imagem_pre['B08'].href, bbox=aoi.total_bounds)
b8A_pre,_ = read(imagem_pre['B8A'].href, bbox=aoi.total_bounds)
b11_pre,_ = read(imagem_pre['B11'].href, bbox=aoi.total_bounds)
b12_pre,_ = read(imagem_pre['B12'].href, bbox=aoi.total_bounds)
_, box_trasform_20m = read(imagem_pre['SCL'].href, bbox=aoi.total_bounds)

In [ ]:
box_trasform_20m

## Transforma as bandas(b04 e b08) de 10m pra 20m - pré-fogo

In [ ]:
#imagens pré-fogo
b04_pre = transforme_20m(b04_pre,b04_pre_transf,ds.crs ) 
b08_pre = transforme_20m(b08_pre,b08_pre_transf,ds.crs ) 

## Leitura das bandas na imagem pós fogo

In [ ]:
#ler imagens pós fogo
b04_pos,b04_pos_transf = read(imagem_pos['B04'].href, bbox=aoi.total_bounds)
b08_pos,b08_pos_transf = read(imagem_pos['B08'].href, bbox=aoi.total_bounds)
b8A_pos,_ = read(imagem_pos['B8A'].href, bbox=aoi.total_bounds)
b11_pos,_ = read(imagem_pos['B11'].href, bbox=aoi.total_bounds)
b12_pos,_ = read(imagem_pos['B12'].href, bbox=aoi.total_bounds)

## Transforma as bandas(b04 e b08) de 10m pra 20m - pós-fogo

In [ ]:
#imagens pós-fogo
b04_pos = transforme_20m(b04_pos,b04_pos_transf,ds.crs ) 
b08_pos = transforme_20m(b08_pos,b08_pos_transf,ds.crs )

## Composição RGB Swir (B12/B08/B04)

In [ ]:
rgb_pre = np.dstack([b12_pre, b08_pre, b04_pre])
rgb_pos = np.dstack([b12_pos, b08_pos, b04_pos])

## 📈 Plot Básico
<hr style="border:1px solid #e69627;">

In [ ]:
# Cria a figura e dois subplots lado a lado (1 linha, 2 colunas)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Primeira imagem: Pré-fogo
axes[0].set_title("Sentinel 2 - Data: 19-08-2024 - Pré Fogo")
axes[0].axis('off')
axes[0].imshow((rgb_pre*255).astype(np.uint8), interpolation='nearest')

# Segunda imagem: Pós-fogo
axes[1].set_title("Sentinel 2 - Data: 24-08-2024 - Pós Fogo")
axes[1].axis('off')
axes[1].imshow((rgb_pos*255).astype(np.uint8), interpolation='nearest')

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()


## 💾 Salvando a Imagem RGB Pré-Fogo no formato GeoTIFF
<hr style="border:1px solid #e69627;">

In [ ]:
# Verifica as dimensões
height, width, num_bands = rgb_pre.shape

with rasterio.open(
    './output/rgb_image_pre.tif',   # nome do arquivo de saída
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=num_bands,            # 3 bandas RGB
    dtype=rgb_pre.dtype,
    crs=ds.crs,                    # sistema de referência (ex.: "EPSG:4326")
    transform=box_trasform_20m         # transform (georreferenciamento)
) as dst:
    for i in range(num_bands):
        dst.write(rgb_pre[:, :, i], i + 1)

print("RGB Pré-Fogo salvo como './output/rgb_image_pre.tif'.")


## 💾 Salvando a Imagem RGB Pós-Fogo no formato GeoTIFF
<hr style="border:1px solid #e69627;">

In [ ]:
# Verifica as dimensões
height, width, num_bands = rgb_pos.shape

with rasterio.open(
    './output/rgb_image_pos.tif',   # nome do arquivo de saída
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=num_bands,            # 3 bandas RGB
    dtype=rgb_pos.dtype,
    crs=ds.crs,                    # sistema de referência (ex.: "EPSG:4326")
    transform=box_trasform_20m         # transform (georreferenciamento)
) as dst:
    for i in range(num_bands):
        dst.write(rgb_pos[:, :, i], i + 1)

print("RGB Pós-Fogo salvo como './output/rgb_image_pos.tif'.")


# <left><span style="color:#cf4c39">Índices 🛰️ Espectrais - NDVI,NBR e NBRSWIR
<hr style="border:1px solid #e69627;">

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">Satélites multiespectrais como o Sentinel-2 registram dados em várias bandas espectrais, incluindo luz visível (Red, Green, Blue) e comprimentos de onda invisíveis como infravermelho próximo (NIR) e infravermelho de ondas curtas (SWIR). Isso abre duas abordagens poderosas para analisar a superfície terrestre.</div><br>

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">Um exemplo popular é o <b>Índice de Vegetação por Diferença Normalizada (NDVI)</b>. Ele aproveita o fato de que a vegetação saudável reflete muito mais radiação NIR do que a luz vermelha. Ao comparar essas duas bandas, podemos mapear a densidade e a saúde da vegetação, conforme é mostrado na figura à esquerda.</div><br> 
<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">   
Da mesma forma, para detectar áreas afetadas pelo fogo, usamos o <b>Índice de Queima Normalizado (NBR)</b>. Este índice compara a refletância NIR e SWIR. A vegetação queimada reflete mais SWIR e menos NIR, tornando o NBR uma ferramenta eficaz para identificar cicatrizes de queimadas, conforme é mostrado na figura à direita abaixo.</div>
<br>
<div style="display: flex; justify-content: center; align-items: flex-start; width: 100%; max-width: 1000px; margin: 0 auto;">
    <div style="margin: 10px; text-align: center;">
        <img src="assets/figura1_espetral.jpg" width="400">
        <p style="text-align: center;"><em>
            Curvas espectrais que mostram a reflectância<br>
            de três tipos de cobertura: vegetação saudável (Healthy Plant),<br>
            vegetação não saudável (Unhealthy Plant) e solo exposto (Soil).
        </em></p>
    </div>
    <div style="margin: 10px; text-align: center;">
        <img src="assets/figura2_espetral.jpg" width="500">
        <p style="text-align: center;"><em>
            Comparação da refletância de áreas queimadas e vegetação <br>
            saudável em todo o espectro,do visível <br>
            ao infravermelho de ondas curtas.
        </em></p>
    </div>
</div>
<br>
A fórmula do NDVI é dada por:

$$
\text{NDVI} = \frac{B08 - B04}{B08 + B04}
$$

A fórmula do NBR é dada por:

$$
\text{NBR} = \frac{B08 - B12}{B08 + B12}
$$

<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;">O índice <b>NBRSWIR (Normalized Burn Ratio SWIR)</b> é uma variação do índice NBR, desenvolvido para aprimorar a detecção de áreas queimadas utilizando imagens do satélite Sentinel-2. Ele foi proposto por Liu et al. (2020) com o objetivo de melhorar a sensibilidade na identificação de áreas afetadas por incêndios, especialmente em ambientes tropicais e subtropicais.</div><br> 
<div style="text-align: justify;  margin-left: 0%; margin-right: 2.5%;"> O <b>NBRSWIR</b> utiliza duas bandas do infravermelho de ondas curtas (SWIR) do Sentinel-2:

- **B11**: SWIR1 (1610 nm)  
- **B12**: SWIR2 (2190 nm)

Essas bandas são sensíveis à **umidade** e à **estrutura da vegetação**, tornando-as eficazes na **detecção de alterações causadas por incêndios**.
<br>

<br>
<b> A fórmula do NBRSWIR é dada por:</b>

$$
\text{NBRSWIR} = \frac{B12 - B11 - 0.02}{B12 + B11 + 0.1}
$$

Esses constantes (−0.02 no numerador, +0.1 no denominador) servem para:

- Reduzir sensibilidade ao ruído, favorecendo a discriminação de áreas queimadas.

- Ajustar o equilíbrio dinâmico entre **SWIR2 (B12)** e **SWIR1 (B11)**, considerando variações de reflectância e condições ambientais.

O **NBRSWIR** ajustado mantém a filosofia do índice original de Liu et al. (2020), mas com uma calibragem específica para Sentinel‑2 e aprimorada para ambientes tropicais ou variáveis.

# <left><span style="color:#cf4c39">Função para calcular os índices espectrais NDVI, NBR e NBRSWIR
<hr style="border:1px solid #e69627;">

In [ ]:
# Geração do Índice NDVI - Exemplo com bandas específicas do Sentinel
# NDVI = (B08 - B04) / (B08 + B04)

def ndvi(b08,b04):
    ndvi=(b08-b04)/(b08+b04)
    return ndvi

# Geração do Índice NBR
# NBR = (B08 - B12) / (B08 + B12)

def nbr(b8A,b12):
    nbr=(b8A-b12)/(b8A+b12)
    return nbr

# Geração do Índice NBRSWIR (Normalized Burn Ratio-SWIR)
# NBRSWIR = (B12 − B11 − 0.02)/( B12 + B11 + 0.1)

def nbrswir(b12,b11):
    nbrswir = (b12 - b11 - 0.02)/(b12 + b11 + 0.1)
    return nbrswir

## Cálculo dos índices espectrais das imagens pré-fogo e pós-fogo

In [ ]:
# calculo dos indices pré fogo
ndvi_pre = ndvi(b08_pre,b04_pre)
nbr_pre = nbr(b8A_pre,b12_pre)
nbrswir_pre = nbrswir(b11_pre,b12_pre)

    
# calculo dos indices pós fogo
ndvi_pos = ndvi(b08_pos,b04_pos)
nbr_pos = nbr(b8A_pos,b12_pos)
nbrswir_pos = nbrswir(b11_pos,b12_pos)

In [ ]:
# Apresenta um pequeno recorte de 3 linhas por 3 colunas de dois índices

print('Valores do NDVI pré')
print(ndvi_pre[:3,:3])

print('\n')

print('Valores do NDVI pós')
print(ndvi_pos[:3,:3])



## 📈 Plot Básico - NDVI
<hr style="border:1px solid #e69627;">

In [ ]:
# Configura a figura com dois subplots (1 linha, 2 colunas)
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Plota imagem NBR pré-fogo
ndvi_pre_plot = axes[0].imshow(ndvi_pre, cmap="RdYlGn", vmin=-0.2, vmax=0.65, interpolation='nearest')
axes[0].set_title("NDVI - Data: 19-08-2024 - Pré Fogo")
axes[0].axis("off")
cbar_pre = plt.colorbar(ndvi_pre_plot, ax=axes[0], fraction=0.03, pad=0.04)
cbar_pre.set_label("NBR Value")

# Plota imagem NBR pós-fogo
ndvi_pos_plot = axes[1].imshow(ndvi_pos, cmap="RdYlGn", vmin=-0.2, vmax=0.65, interpolation='nearest')
axes[1].set_title("NDVI - Data: 24-08-2024 - Pós Fogo")
axes[1].axis("off")
cbar_pos = plt.colorbar(ndvi_pos_plot, ax=axes[1], fraction=0.03, pad=0.04)
cbar_pos.set_label("NBR Value")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

## 📈 Plot Básico - NBR
<hr style="border:1px solid #e69627;">

In [ ]:
# Configura a figura com dois subplots (1 linha, 2 colunas)
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Plota imagem NBR pré-fogo
nbr_pre_plot = axes[0].imshow(nbr_pre, cmap="RdYlGn", vmin=-0.35, vmax=0.35, interpolation='nearest')
axes[0].set_title("NBR - Data: 19-08-2024 - Pré Fogo")
axes[0].axis("off")
cbar_pre = plt.colorbar(nbr_pre_plot, ax=axes[0], fraction=0.03, pad=0.04)
cbar_pre.set_label("NBR Value")

# Plota imagem NBR pós-fogo
nbr_pos_plot = axes[1].imshow(nbr_pos, cmap="RdYlGn", vmin=-0.35, vmax=0.35, interpolation='nearest')
axes[1].set_title("NBR - Data: 24-08-2024 - Pós Fogo")
axes[1].axis("off")
cbar_pos = plt.colorbar(nbr_pos_plot, ax=axes[1], fraction=0.03, pad=0.04)
cbar_pos.set_label("NBR Value")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

## 📈 Plot Básico - NBRSWIR
<hr style="border:1px solid #e69627;">

In [ ]:
# Configura a figura com dois subplots (1 linha, 2 colunas)
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Plota imagem NBR pré-fogo
nbrswir_pre_plot = axes[0].imshow(nbrswir_pre, cmap="RdYlGn", vmin=-0.15, vmax=0.15, interpolation='nearest')
axes[0].set_title("NBRSWIR - Data: 19-08-2024 - Pré Fogo")
axes[0].axis("off")
cbar_pre = plt.colorbar(nbrswir_pre_plot, ax=axes[0], fraction=0.03, pad=0.04)
cbar_pre.set_label("NBRSWIR Value")

# Plota imagem NBR pós-fogo
nbrswir_pos_plot = axes[1].imshow(nbrswir_pos, cmap="RdYlGn", vmin=-0.15, vmax=0.15, interpolation='nearest')
axes[1].set_title("NBRSWIR - Data: 24-08-2024 - Pós Fogo")
axes[1].axis("off")
cbar_pos = plt.colorbar(nbrswir_pos_plot, ax=axes[1], fraction=0.03, pad=0.04)
cbar_pos.set_label("NBRSWIR Value")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

# <left><span style="color:#cf4c39">Calculando a área queimada com dNBRSWIR 🧮
<hr style="border:1px solid #e69627;">

Agora, será utilizada uma téncnica simples de detecção de mudanças temporais que é o calculo de diferença de cada Índice antes e depois do incêndio.

## Cálculo da diferença relativa dos índices espectrais das imagens pré-fogo e pós-fogo

In [ ]:
# cálculo da diferença relativa dos índices espectrais
ndvi_dif = ndvi_pre - ndvi_pos

nbr_dif = nbr_pre - nbr_pos

nbrswir_dif = nbrswir_pre - nbrswir_pos

In [ ]:
print('Apresenta um pequeno recorte de 3 linhas por 3 colunas de dois índices \n')

print('Valores do NDVI pré')
print(ndvi_pre[:3,:3])

print('\n')

print('Valores do NDVI pós')
print(ndvi_pos[:3,:3])

print('\n')

print('Valores de Diferenças do NDVI')
print(ndvi_dif[:3,:3])

---

In [ ]:
# Configura a figura com dois subplots (1 linha, 3 colunas)
fig, axes = plt.subplots(1, 3, figsize=(14, 7))

# Plota diferença NDVI
ndvi_dif_plot = axes[0].imshow(ndvi_dif, cmap="PuOr", vmin=-0.15, vmax=0.25, interpolation='nearest')
axes[0].set_title("NDVI Diferença (Pré - Pós Fogo)")
axes[0].axis("off")
cbar_ndvi = plt.colorbar(ndvi_dif_plot, ax=axes[0], fraction=0.03, pad=0.04)
cbar_ndvi.set_label("NDVI Difference")

# Plota diferença NBR
nbr_dif_plot = axes[1].imshow(nbr_dif, cmap="PuOr", vmin=-0.15, vmax=0.25, interpolation='nearest')
axes[1].set_title("NBR Diferença (Pré - Pós Fogo) ")
axes[1].axis("off")
cbar_nbr = plt.colorbar(nbr_dif_plot, ax=axes[1], fraction=0.03, pad=0.04)
cbar_nbr.set_label("NBR Difference")

# Plota diferença NBR
nbrswir_dif_plot = axes[2].imshow(nbrswir_dif, cmap="PuOr", vmin=-0.15, vmax=0.25, interpolation='nearest')
axes[2].set_title("NBR_Swir  Diferença (Pré - Pós Fogo) ")
axes[2].axis("off")
cbar_nbr = plt.colorbar(nbrswir_dif_plot, ax=axes[2], fraction=0.03, pad=0.04)
cbar_nbr.set_label("NBR_Swir Difference")

# Ajusta o layout para evitar sobreposição
plt.tight_layout()
plt.show()

<br>
<hr>

## 💾 Salvando o índice NBRSWIR Diferença no formato GeoTIFF
<hr style="border:1px solid #e69627;">

In [ ]:
# Exemplo básico — ajuste conforme seu contexto
height, width = nbr_dif.shape

# Abre o arquivo para escrita
with rasterio.open(
    './output/nbrswir_dif.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,  # número de bandas
    dtype=nbr_dif.dtype,
    crs=ds.crs,  # ex: 'EPSG:4326' ou objeto rasterio.crs.CRS
    transform=box_trasform_20m,
) as dst:
    dst.write(nbrswir_dif, 1)  # escreve na banda 1
    
print("NBRSWIR Diferença  salvo como './output/nbrswir_dif.tif'.")

# <left><span style="color:#cf4c39">Resultados 📊
<hr style="border:1px solid #e69627;">

In [ ]:
# Utiliza o método where do numpy para criar uma máscara de 0 e 1

threshold = 0.09 # limiar que veio do qgis

dnbrswir_thres_0_1= np.where(nbrswir_dif>=threshold, 1, 0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(dnbrswir_thres_0_1)
plt.title(" (dNBRswir) com Threshold 0.09")
plt.show()

In [ ]:


# Cria a figura com dois subplots de igual largura
fig, axes = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [1, 1]})

# Primeiro plot (dNBRswir)
dnbrswir_plot = axes[0].imshow(nbrswir_dif, cmap="PuOr", vmin=-0.15, vmax=0.25, interpolation='nearest')
axes[0].set_title("Difference of Normalized Burn Ratio SWIR (dNBRswir)")
axes[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

# Colorbar para o primeiro plot
cbar = plt.colorbar(dnbrswir_plot, ax=axes[0], fraction=0.03, pad=0.04)
cbar.set_label("dNBRswir Value")

# Segundo plot (área mascarada)
nbrswir_plot = axes[1].imshow(dnbrswir_thres_0_1, cmap='gray_r')
axes[1].set_title("dNBRswir masked area after touch up")
axes[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.tight_layout()
plt.show()


## Converte a Máscara de formato Raster para Vector e salva em Shapefile

In [ ]:

from rasterio.features import shapes
from shapely.geometry import shape

# Converte a máscara para int (0 and 1)
mask_burn = dnbrswir_thres_0_1.astype(np.uint8)

# Converte para poligos
results = shapes(mask_burn, mask=mask_burn == 1, transform=box_trasform_20m)

# Convert para um objeto de geometria da biblioteca shapely
geoms = [shape(geom) for geom, val in results if val == 1]

# Cria um GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=geoms, crs=ds.crs)

# Optional: Add a 'burnt' column for clarity
gdf["queimada"] = 1

# Save to GeoPackage or Shapefile em UTM
gdf.to_file("./output/area_queimada_dnbrswir_utm.shp", driver= "ESRI Shapefile")

# Transforma para EPSG:4326
gdf = gdf.to_crs("EPSG:4326")

# Optional: Add a 'burnt' column for clarity
gdf["queimada"] = 1

# Save to GeoPackage or Shapefile em UTM
gdf.to_file("./output/area_queimada_dnbrswir_wgs84.shp", driver= "ESRI Shapefile")

### Criamos um gráfico que mostra os incêndios detectados sobre a cicatriz de queimada. Os focos de calor detectados são agrupados por dia para ver onde o fogo começou e como se espalhou.

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Read the DBF file
try:
    gdf = gpd.read_file('./data/focos/focos_1908_2408_2024.dbf')
except FileNotFoundError:
    print("Error: DBF file not found at './data/focos/focos_1908_2408_2024.dbf'. Please check the file path.")
    exit()

# Step 2: Select relevant columns and ensure they're named correctly
df = gdf[['longitude', 'latitude', 'data_hora_']].copy()  # Use copy to avoid SettingWithCopyWarning
df = df.dropna(subset=['longitude', 'latitude', 'data_hora_'])

# Step 2.5: Convert data_hora_ to integer days starting from 0
try:
    df['data_hora_'] = pd.to_datetime(df['data_hora_'])
except ValueError:
    print("Error: Unable to parse 'data_hora_' column as datetime. Please check the format.")
    exit()
df['days'] = (df['data_hora_'] - df['data_hora_'].min()).dt.days

# Step 3: Set up the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Step 4: Read and plot burnt area polygons
try:
    polygons = gpd.read_file('./output/area_queimada_dnbrswir_wgs84.shp')
    # Transformar para EPSG:4326
    # polygons = polygons.to_crs("EPSG:4326") 
    polygons.plot(ax=ax, color='red', edgecolor='black', linewidth=0.5, alpha=1)
        
except FileNotFoundError:
    print("Warning: Shapefile './input/burnt_area.shp2' not found. Skipping polygon plotting.")

# Step 5: Scatter plot with colormap
scatter = ax.scatter(
    df['longitude'],
    df['latitude'],
    c=df['days'],
    cmap='viridis',
    s=10,
    edgecolor='k',
    linewidth=0.2
)

# Step 6: Colorbar
cbar = plt.colorbar(scatter, ax=ax, fraction=0.03, pad=0.04, label='Day')
cbar.ax.tick_params(labelsize=8)

# Step 7: Set axis labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Focos de Calor por Dia com Área Queimada', fontsize=16)

# Step 8: Set axis limits to the provided bbox

if not polygons.empty:
    minx, miny, maxx, maxy = polygons.total_bounds
    expanded_bbox = box(minx - 0.02, miny - 0.02, maxx + 0.02, maxy + 0.02)  # cria polígono
    minx, miny, maxx, maxy = expanded_bbox.bounds  # extrai bbox do polígono
    
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

plt.tight_layout()
plt.show()

### Evolução dos focos de calor detectações no intervalo de 20/08 a 24/08 de 2024

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df['data_hora_'] = pd.to_datetime(df['data_hora_'])
df['data'] = df['data_hora_'].dt.date
df['hora'] = df['data_hora_'].dt.hour

heatmap_data = df.groupby(['data', 'hora']).size().unstack(fill_value=0)
totais_por_dia = heatmap_data.sum(axis=1)

fig, ax = plt.subplots(figsize=(13, 5))

sns.heatmap(heatmap_data, cmap='magma', annot=True, fmt='d', ax=ax, cbar_kws={'shrink': 0.7})

ax.set_title('Número de Detecções por Dia e Hora')
ax.set_xlabel('Hora do Dia')
ax.set_ylabel('Data')

# Para alinhar os totais ao lado das linhas do heatmap
for y, total in enumerate(totais_por_dia):
    ax.text(heatmap_data.shape[1] + 0.3, y + 0.5, str(total), va='center', ha='left', fontsize=9, color='black')

# Ajustar limites do eixo x para dar espaço para os números
ax.set_xlim(0, heatmap_data.shape[1] + 1.5)

plt.tight_layout()
plt.show()


In [ ]:
# Total de detecções por dia
total_por_dia = df.groupby('data').size().reset_index(name='total_deteccoes')

print(total_por_dia)
